In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
t=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
t.head()

In [ ]:
print("Shape of Training Data")
print(t.shape)
print("\n")
print(t.info())
print("################")
t['type'].value_counts()



In [ ]:
te=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
te.head()

In [ ]:
te.shape
sns.pairplot(data=t,hue='type')


In [ ]:
_ = sns.heatmap(t.corr(), annot = True, fmt = ".2f", cmap = 'YlGnBu')

In [ ]:
g = sns.FacetGrid(pd.melt(t, id_vars='type', value_vars = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul']), col = 'type')
g = g.map(sns.boxplot, 'value', 'variable', palette = 'muted')

In [ ]:
t=pd.concat([t,pd.get_dummies(t['color'])],axis=1)
t.drop('color',axis=1,inplace=True)
t.head()

In [ ]:
te=pd.concat([te,pd.get_dummies(te['color'])],axis=1)
te.drop('color',axis=1,inplace=True)
te.head()

In [ ]:
X=t.drop(['id','type'],axis=1)
y=pd.get_dummies(t['type'])
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

from tensorflow import keras

from keras.layers import Dense,Dropout
from keras.models import Sequential



In [ ]:
model=Sequential()
model.add(Dense(64,input_shape=(X.shape[1],)))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
train=model.fit(x=X_train,y=y_train,batch_size=16,epochs=10,verbose=2,validation_data=(X_test,y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.plot(train.history['accuracy'],'r',label='Training accuracy')
plt.plot(train.history['val_accuracy'],'b',label='Validation accuracy')
plt.legend()

In [ ]:
pred=model.predict(te.drop('id',axis=1))

pred_final=[np.argmax(i) for i in pred]

submission = pd.DataFrame({'id':te['id'], 'type':pred_final})
submission.head()



In [ ]:
submission['type'].replace(to_replace=[0,1,2],value=['Ghost','Ghoul','Goblin'],inplace=True)
submission.head()

In [ ]:
submission.to_csv('../working/submission.csv', index=False)